In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Imports

In [ ]:
from typing import Optional

from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langsmith import traceable
from pydantic import BaseModel
from typing_extensions import TypedDict

load_dotenv()

## Vanilla workflow

In [ ]:
class State(BaseModel):
    topic: str
    table_of_contents: Optional[str] = None
    content: Optional[str] = None
    revised_content: Optional[str] = None


model = ChatOpenAI(model="gpt-4.1-mini")


@traceable
def generate_table_of_contents(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, you will generate the table of contents for a short article."
        ),
        HumanMessage(
            content=f"Generate the table of contents of an article about {state.topic}"
        ),
    ]
    return model.invoke(messages).content


@traceable
def generate_article_content(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic and a table of contents, you will generate the content of the article."
        ),
        HumanMessage(
            content=f"Generate the content of an article about {state.topic} with the following table of contents: {state.table_of_contents}"
        ),
    ]
    return model.invoke(messages).content


@traceable
def revise_article_content(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, a table of contents and a content, you will revise the content of the article to make it less than 1000 characters."
        ),
        HumanMessage(
            content=f"Revise the content of an article about {state.topic} with the following table of contents: {state.table_of_contents} and the following content:\n\n{state.content}"
        ),
    ]
    return model.invoke(messages).content


@traceable
def generate_article(topic: str) -> State:
    article = State(topic=topic)
    article.table_of_contents = generate_table_of_contents(article)
    article.content = generate_article_content(article)
    if len(article.content) > 1000:
        article.revised_content = revise_article_content(article)
    return article


article = generate_article("Artificial Intelligence")
article

## LangGraph implementation

In [ ]:
class State(TypedDict):
    topic: str
    table_of_contents: str
    content: str
    revised_content: str


def generate_table_of_contents_lg(state: State) -> dict:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, you will generate the table of contents for a short article."
        ),
        HumanMessage(
            content=f"Generate the table of contents of an article about {state['topic']}"
        ),
    ]
    return {"table_of_contents": model.invoke(messages).content}


def generate_article_content_lg(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic and a table of contents, you will generate the content of the article."
        ),
        HumanMessage(
            content=f"Generate the content of an article about {state['topic']} with the following table of contents: {state['table_of_contents']}"
        ),
    ]
    return {"content": model.invoke(messages).content}


def check_article_content(state: State) -> str:
    if len(state["content"]) > 1000:
        return "Fail"
    return "Pass"


def revise_article_content_lg(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, a table of contents and a content, you will revise the content of the article to make it less than 1000 characters."
        ),
        HumanMessage(
            content=f"Revise the content of an article about {state['topic']} with the following table of contents: {state['table_of_contents']} and the following content:\n\n{state['content']}"
        ),
    ]
    return {"revised_content": model.invoke(messages).content}


workflow = StateGraph(State)

workflow.add_node("generate_table_of_contents", generate_table_of_contents_lg)
workflow.add_node("generate_article_content", generate_article_content_lg)
workflow.add_node("revise_article_content", revise_article_content_lg)

workflow.add_edge(START, "generate_table_of_contents")
workflow.add_edge("generate_table_of_contents", "generate_article_content")
workflow.add_conditional_edges(
    source="generate_article_content",
    path=check_article_content,
    path_map={"Fail": "revise_article_content", "Pass": END},
)
workflow.add_edge("revise_article_content", END)

chain = workflow.compile()

display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
state = chain.invoke({"topic": "Artificial Intelligence"})

print("Table of contents:")
print(state["table_of_contents"])
print("\n--- --- ---\n")
if "content" in state and state["content"] is not None:
    print("Article content:")
    print(state["content"])
    print("\n--- --- ---\n")

if "revised_content" in state:
    print("Revised article content:")
    print(state["revised_content"])
else:
    print("Article passed quality gate - no revised content detected!")

# Exercise
Build a workflow to generate recipes of food. If the recipe includes more than 5 ingredients, revise the recipe to try to exclude some ingredients if possible.

You should be able to compare the initial and revised recipe and ingredients.

In [ ]:
class State(TypedDict):
    recipe_request: str
    ingredients: list[str]
    recipe_content: str
    revised_ingredients: list[str]
    revised_recipe_content: str


class Ingredients(BaseModel):
    ingredients: list[str]


class RevisedRecipe(BaseModel):
    ingredients: list[str]
    revised_recipe_content: str


def get_ingredients(state: State) -> dict:
    model_with_structure = model.with_structured_output(Ingredients)
    messages = [
        SystemMessage(
            content="You are an expert chef. Provided with a recipe request, you will generate a list of ingredients."
        ),
        HumanMessage(
            content=f"Generate a list of ingredients for the following recipe request: {state['recipe_request']}"
        ),
    ]
    return {"ingredients": model_with_structure.invoke(messages).ingredients}


def generate_recipe_content(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert chef. Provided with a recipe request and a list of ingredients, you will generate a recipe."
        ),
        HumanMessage(
            content=f"Write a recipe for {state['recipe_request']} with the following ingredients:\n\n{state['ingredients']}"
        ),
    ]
    return {"recipe_content": model.invoke(messages).content}


def revise_recipe_content(state: State) -> str:
    model_with_structure = model.with_structured_output(RevisedRecipe)
    messages = [
        SystemMessage(
            content="You are an expert chef. Provided with a recipe request, a list of ingredients and a recipe, you should try to remove any unnecessary ingredients. If possible, you should aim for a recipe with less than 5 ingredients."
        ),
        HumanMessage(
            content=f"Revise the recipe for {state['recipe_request']} with the following ingredients:\n\n{state['ingredients']}\n\nand the following recipe:\n\n{state['recipe_content']}"
        ),
    ]
    response = model_with_structure.invoke(messages)
    return {
        "revised_recipe_content": response.revised_recipe_content,
        "revised_ingredients": response.ingredients,
    }


def check_recipe_content(state: State) -> str:
    if len(state["ingredients"]) > 5:
        return "Fail"
    return "Pass"


workflow = StateGraph(State)

workflow.add_node("get_ingredients", get_ingredients)
workflow.add_node("generate_recipe_content", generate_recipe_content)
workflow.add_node("revise_recipe_content", revise_recipe_content)

workflow.add_edge(START, "get_ingredients")
workflow.add_edge("get_ingredients", "generate_recipe_content")
workflow.add_conditional_edges(
    source="generate_recipe_content",
    path=check_recipe_content,
    path_map={"Fail": "revise_recipe_content", "Pass": END},
)

chain = workflow.compile()

display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
chain.invoke({"recipe_request": "A recipe for a healthy breakfast"})